In [1]:
from scipy.stats import norm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor
import random
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import itertools

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('aaindex_dict.pickle', 'rb') as handle:
    aa_index = pickle.load(handle)

In [3]:
class do_ml(object):

    def __init__(self, df, features_AS, target, encoding, components=11, verbose=True, model_type="gp", testing=True):

        self.seed_everything()

        self.df = df
        self.features_AS = features_AS
        self.target = target

        self.encoding = encoding
        self.components = components

        self.prepare_encoding(self.components)

        self.encoded_as_df = self.make_t_scale_df(self.df[self.features_AS])
        self.y = self.df[self.target]

        self.all_as = ["R", "H", "K", "D", "E", "S", "T", "N", "Q",
                       "C", "G", "P", "A", "V", "I", "L", "M", "F", "W", "Y"]
        self.len_cats = len(features_AS)

        all_as_combs = list(itertools.product(
            self.all_as, repeat=self.len_cats))

        self.all_as_df = pd.DataFrame(all_as_combs, columns=self.features_AS)
        self.all_as_df_encoded = self.make_t_scale_df(
            self.all_as_df[self.features_AS])

        self.verbose = verbose
        self.model_type = model_type

        self.testing = testing

    def prepare_encoding(self, components):

        pc = PCA(n_components=components)
        pc = pc.fit_transform(np.stack(self.encoding.values()))

        encoding_df = pd.DataFrame({"AS": list(self.encoding.keys())})

        for i in range(pc.shape[1]):
            encoding_df["f_"+str(i)] = pc[:, i]

        for t in encoding_df.columns[1:]:
            sclr = StandardScaler()
            encoding_df[t] = sclr.fit_transform(
                np.array(encoding_df[t]).reshape((-1, 1)))

        self.encoding_df = encoding_df

    def make_t_scale_df(self, df):

        cols = df.columns
        for t, i in enumerate(cols):
            df = pd.merge(df, self.encoding_df, how="left",
                          left_on=i, right_on="AS")
            df.drop("AS", axis=1, inplace=True)
            keep_cols = self.encoding_df.columns[1:]
            df.rename(columns={x: str(x) + "_" + str(t + 1)
                      for x in keep_cols}, inplace=True)
        df.drop(cols, axis=1, inplace=True)

        return df

    def update_dfs(self, components):
        self.prepare_encoding(components)

        self.encoded_as_df = self.make_t_scale_df(self.df[self.features_AS])
        self.all_as_df_encoded = self.make_t_scale_df(
            self.all_as_df[self.features_AS])

    def predict(self, n_splits=10):

        self.target = np.log1p(self.y)

        n_splits = 10

        cv = KFold(n_splits=n_splits)

        self.oof_pred = np.zeros(len(self.encoded_as_df))
        self.oof_all_as_pred = np.zeros(len(self.all_as_df_encoded))

        for t, (train_idx, val_idx) in enumerate(cv.split(self.encoded_as_df)):
            x_train, y_train = self.encoded_as_df.iloc[train_idx], self.target[train_idx]
            x_val, y_val = self.encoded_as_df.iloc[val_idx], self.target[val_idx]

            #kernel =  RBF(length_scale_bounds = "fixed")

            if self.model_type == "gp":
                kernel = Matern(length_scale=1., nu=1.5)
                lreg = GaussianProcessRegressor(kernel)

            elif self.model_type == "rf":
                lreg = RandomForestRegressor()

            lreg.fit(x_train, y_train)
            yhat = lreg.predict(x_val)

            if not self.testing:
                if len(self.all_as_df_encoded) <= 20**3:
                    yhat_pred = lreg.predict(self.all_as_df_encoded).astype(
                        "float16")  # np.ones(len(self.all_as_df_encoded))

                else:
                    yhat_pred = []
                    for sub_df in np.array_split(self.all_as_df_encoded, 25):
                        # print(sub_df.shape)
                        yhat_pred_sub = lreg.predict(sub_df).astype("float16")
                        yhat_pred = np.concatenate([yhat_pred, yhat_pred_sub])

            else:
                yhat_pred = np.ones(len(self.all_as_df_encoded))

            yhat = np.expm1(yhat.clip(0, 10))
            yhat_pred = np.expm1(yhat_pred.clip(0, 10))

            self.oof_pred[val_idx] = yhat
            self.oof_all_as_pred += yhat_pred / n_splits

            r2 = r2_score(np.expm1(y_val), yhat)
            mse = mean_squared_error(np.expm1(y_val), yhat)

            if self.verbose:
                print(f"{r2}")

        print(f"final r2: {r2_score(np.expm1(self.target),self.oof_pred)}")
        print(
            f"final mse: {mean_squared_error(np.expm1(self.target),self.oof_pred)}")
        self.final_r2 = r2_score(np.expm1(self.target), self.oof_pred)

    def plot(self):

        lreg = LinearRegression()

        lreg.fit(self.oof_pred.reshape(-1, 1), self.y.values.reshape(-1, 1))

        x = np.linspace(0, self.oof_pred.max() +
                        self.oof_pred.max()*0.1, 300).reshape(-1, 1)
        y = lreg.predict(x)

        plt.figure(figsize=(15, 10))
        plt.scatter(self.oof_pred, self.y)
        plt.xlabel("predicted values", fontsize=15)
        plt.ylabel("measured values", fontsize=15)
        plt.plot(x, y, c="r")

    @staticmethod
    def seed_everything(seed=0):
        random.seed(seed)
        np.random.seed(seed)

In [4]:
data = pd.read_excel("library16_data.xlsx")

In [5]:
libraries_mutations = {}
for u in data.library.unique():
    libs = []
    for c in [97, 174, 238, 241, 242, 245]:
        am = len(data.loc[data.library == u, c].value_counts())

        if am > 4:
            libs.append(c)

    libraries_mutations[u] = libs

In [6]:
libraries_mutations

{13: [174, 238, 241, 242, 245],
 12: [174, 238, 241],
 11: [238, 241, 242],
 10: [241, 242, 245],
 9: [174, 242, 245],
 'single': [97, 174, 238, 241, 242, 245]}

In [7]:
feature_cols = libraries_mutations[13]
target_cols = "FO_WT"

test_lin = data.loc[data.library.isin([9, 10, 12, 13])].reset_index(drop=True)
test_lin["FO_WT"] = test_lin.groupby("variant")["FO_WT"].transform("mean")

test_lin = test_lin.drop_duplicates(subset="variant").reset_index(drop=True)
len_before = len(test_lin)

as_test = ["R", "H", "K", "D", "E", "S", "T", "N", "Q",
           "C", "G", "P", "A", "V", "I", "L", "M", "F", "W", "Y"]
test_lin = test_lin[test_lin[feature_cols].apply(
    lambda x: all(x.isin(as_test)), axis=1)].reset_index(drop=True)
len_after = len(test_lin)

print(len_after)

test_lin[target_cols] = test_lin[target_cols].apply(lambda x: max(0, x))

t = do_ml(test_lin, feature_cols, target_cols, aa_index,
          verbose=True, components=13, model_type="gp", testing=False)
t.predict()

df5_only = list(t.df[t.df["library"] == 13].index)
df5_r2 = r2_score(np.expm1(t.target[df5_only]), t.oof_pred[df5_only])

print("*" * 20, df5_r2)

2453
0.517710517749248
0.5444651276400366
0.5731127269649066
0.7826878388432748
0.804462778627905
0.8126885925182359
0.6837524577824216
0.7117470587591613
0.5307751337829878
0.8391879096053694
final r2: 0.7696547119752704
final mse: 0.07484251153982062
******************** 0.5414771472857689


In [8]:
all5df = t.all_as_df
all5df["preds"] = t.oof_all_as_pred

In [9]:
all5df = all5df.sort_values("preds", ascending=False)[
    :10000].reset_index(drop=True)

In [10]:
all5df["variant"] = all5df[all5df.columns[:-1]
                           ].apply(lambda x: "".join(x), axis=1)

In [11]:
all5df["variant"] = all5df.variant.apply(lambda x: "W"+x)

In [12]:
all5df = all5df.loc[~all5df.variant.isin(
    test_lin.variant)].reset_index(drop=True)

In [13]:
all5df.to_excel("lib_16_predictions.xlsx", index=False)